# Make LDCs based on co-added PHOENIX spectrum

In [2]:
%matplotlib qt

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from astropy.io import fits
import astropy.units as u
from glob import glob
import get_lds
from prettyplots import pretty_axis
import scipy.interpolate as si
import json

In [4]:
def get100_PHOENIX(wavelengths, I, new_mu, idx_new):
    mu100 = np.arange(0.01, 1.01, 0.01)
    I100 = np.zeros((len(wavelengths),len(mu100)))
    for i in range(len(wavelengths)):
        # Cubic splines (k=3), interpolation through all points (s=0) ala CB11.
        II = si.UnivariateSpline(new_mu, I[i,idx_new], s=0, k=3)
        I100[i] = II(mu100)
    return mu100, I100

def get_ldcs(min_w,max_w):
    infiles = glob("./PHOENIX/*08800*SPECINT*.fits")
    #infiles = [infiles[-1]]
    I_array = np.empty(shape=(len(infiles),25500,78))
    i = 0
    for file in infiles:
        with fits.open(file) as hdul:
            data = hdul[0].data
            mu = hdul[1].data
            CDELT1 = hdul[0].header['CDELT1']
            CRVAL1 = hdul[0].header['CRVAL1']
            
            wavelengths = np.arange(data.shape[1]) * CDELT1 + CRVAL1
            I = data.transpose()
            I_array[i] = I
            i += 1
    
    I = np.mean(I_array,axis=0)
    
    # Get response function
    response_file = "./COS_G160M_FLUXTAB_SENSITIVITY.txt"
    
    min_w,max_w,S_wav,S_res = get_lds.get_response(min_w,max_w,response_file)
    
    # Integrate intensity w/ response function
    I0 = get_lds.integrate_response_PHOENIX(wavelengths,I,mu,S_res,S_wav,correction=True,interpolation_order=1)
    
    # Obtain correction due to spherical extension. First, get r_max:
    r, fine_r_max = get_lds.get_rmax(mu, I0)
    
    # Now get r for each intensity point and leave out those that have r>1:
    new_r = r/fine_r_max
    idx_new = new_r <= 1.0
    new_r = new_r[idx_new]
    # Reuse variable names:
    mu = np.sqrt(1.0-(new_r**2))
    I0 = I0[idx_new]

    mu, I100 = get_lds.get100_PHOENIX(wavelengths, I, mu, idx_new)
    I0 = get_lds.integrate_response_PHOENIX(wavelengths, I100, mu,S_res, S_wav, correction=True, interpolation_order=1)
    idx = mu >= 0.0
    
    # Now compute each LD law:
    c1, c2, c3, c4 = get_lds.fit_non_linear(mu, I0)
    a = get_lds.fit_linear(mu[idx], I0[idx])
    u1, u2 = get_lds.fit_quadratic(mu[idx], I0[idx])
    b1, b2, b3 = get_lds.fit_three_parameter(mu[idx], I0[idx])
    l1, l2 = get_lds.fit_logarithmic(mu[idx], I0[idx])
    e1, e2 = get_lds.fit_exponential(mu[idx], I0[idx])
    s1, s2 = get_lds.fit_square_root(mu[idx], I0[idx])
    LDC = a, u1, u2, b1, b2, b3, c1, c2, c3, c4, l1, l2, e1, e2, s1, s2

    return LDC,mu,I0

def ldc_quadratic(mu,u):
    I = 1-u[0]*(1-mu)-u[1]*(1-mu)**2
    return I

def ldc_nonlinear(mu,c):
    I = 1-np.sum([c[i-1]*(1-mu**(i/2)) for i in range(1,5)],axis=0)
    return I

def ldc_threeparam(mu,b):
    I = 1-b[0]*(1-mu)-b[1]*(1-mu**(3/2)-b[2]*(1-mu**2))
    return I

In [5]:
ldcs_a,mu_a,I0_a = get_ldcs(1600,1760)
ldcs_b,mu_b,I0_b = get_ldcs(1410,1580)

c_a,c_b = ldcs_a[6:10],ldcs_b[6:10]
u_a,u_b = ldcs_a[1:3],ldcs_b[1:3]
b_a,b_b = ldcs_a[3:6],ldcs_b[3:6]

print("Segment A LDCs")
print("Quadratic: ",u_a)
print("Non-linear: ",c_a,end="\n\n----------\n\n")
print("Segment B LDCs")
print("Quadratic: ",u_b)
print("Non-linear: ",c_b)

intensity_quad_a,intensity_quad_b = ldc_quadratic(mu_a,u_a),ldc_quadratic(mu_b,u_b)
quad_a,quad_b = ldc_quadratic(mu_a,(1.76193636, -0.77558164)),ldc_quadratic(mu_b,(2.44873077, -1.52697716))
quad_a_MCMC,quad_b_MCMC = ldc_quadratic(mu_a,(0.497048,-1.71372)),ldc_quadratic(mu_b,(2.5845121, -1.867060))
intensity_nonlinear_a,intensity_nonlinear_b = ldc_nonlinear(mu_a,c_a),ldc_nonlinear(mu_b,c_b)
intensity_threeparam_a,intensity_threeparam_b = ldc_threeparam(mu_a,b_a),ldc_threeparam(mu_b,b_b)

Segment A LDCs
Quadratic:  (np.float64(1.7749253509572227), np.float64(-0.7912525208852434))
Non-linear:  (np.float64(0.5301500175169972), np.float64(-1.8995689043902635), np.float64(2.916057873741457), np.float64(-0.5313215791395192))

----------

Segment B LDCs
Quadratic:  (np.float64(2.3846545351270416), np.float64(-1.4527589443123696))
Non-linear:  (np.float64(0.12886529926670323), np.float64(0.3148215592975764), np.float64(-2.059035255816571), np.float64(2.618903614107086))


In [5]:
# Fe II intensity fit
ldc_fe, mu_fe, I_fe = get_ldcs(min_w=1606.84155, max_w=1610.05845)
ldc_fe = ldc_fe[6:10]
intensity_fit = ldc_nonlinear(mu_fe,ldc_fe)

# Model LDCs from batman.ipynb
model = 4

match model:
    # BATMAN uniform
    case 1:
        ldcs = [0.47491752,-1.67688194,2.82185299,-0.09055813]
        label='BATMAN uniform'
    # BATMAN gaussian
    case 2:
        ldcs = [0.30023484,-1.74519917,2.95901681,0.02688968]
        label = 'BATMAN gaussian'
    # PyTransit uniform
    case 3:
        ldcs = [0.44333106,-2.11230645,2.56604699,0.07421745]
        label = 'PyTransit uniform'
    # PyTransit gaussian
    case 4:
        ldcs = [0.59993584,-2.17445874,2.75924478,-0.01660394]
        label = 'PyTransit gaussian'
        
mcmc_fit = ldc_nonlinear(mu_fe,ldcs)

fig,ax = plt.subplots(1,1,tight_layout=True)
pretty_axis(ax)
ax.plot(mu_fe,I_fe,lw=2,label='PHOENIX intensity')
ax.plot(mu_fe,intensity_fit,label='EJ15 fit')
ax.plot(mu_fe,mcmc_fit,label=label)
ax.legend(fontsize=14)
ax.set_xlabel(r'$\mu$',fontsize=18)
ax.set_ylabel(r'I($\mu$) / I(1)',fontsize=18)

Text(0, 0.5, 'I($\\mu$) / I(1)')

In [5]:
wave_ranges = {
    '1400-1600':[1400,1600],
    '1600-1800':[1600,1800],
    '4000-9000':[4000,9000],
}

linestyles = ['-','--','-.']

fig,ax = plt.subplots(1,1,tight_layout=True,figsize=(10,8))
pretty_axis(ax)

for i,wave in enumerate(wave_ranges):
    w_max,w_min = wave_ranges[wave]
    ldc,mu,I = get_ldcs(w_max,w_min)
    c = ldc[6:10]
    intensity = ldc_nonlinear(mu,c)
    print(wave_ranges[wave])
    print(c,end='\n-------\n')
    ax.plot(mu,intensity,lw=2,label=wave+' Å')

ax.legend(fontsize=15)
ax.set_xlabel(r'$\mu=\sqrt{1-r^2}$',fontsize=18)
ax.set_ylabel(r'I($\mu$) / I(1)',fontsize=18)

[1400, 1600]
(np.float64(0.21335080867633954), np.float64(-0.1537730860891631), np.float64(-1.0622477762563052), np.float64(2.0090048259977618))
-------
[1600, 1800]
(np.float64(0.489208604424788), np.float64(-1.6731735018199676), np.float64(2.71207780575705), np.float64(-0.5189304019180775))
-------
[4000, 9000]
(np.float64(1.2869596424408472), np.float64(-2.1723549474816717), np.float64(2.7284373950029748), np.float64(-1.1212673949493246))
-------


Text(0, 0.5, 'I($\\mu$) / I(1)')

In [7]:
wavelengths = {
    'Fe II 1608':1608.45,
    'Al II 1671':1670.74,
    'Si II 1533':1533.45,
    'Si II 1526':1526.72,
    'S I 1425':1425.03,
    'S I 1433':1433.28,
    'N I 1494':1493.65,
    'C I 1657':1657,
}

ldc_dict = {}

fig,ax = plt.subplots(1,1,tight_layout=True)
pretty_axis(ax)

ax.plot(mu_b,intensity_nonlinear_b,label='1400-1600 Å',lw=3)
ax.plot(mu_a,intensity_nonlinear_a,label='1600-1800 Å',lw=3)

for i,ion in enumerate(wavelengths):
    # Get wavelength center
    wave = wavelengths[ion]
    # Calculate wavelength range
    dlam = 300/3e5*wave
    wave_range = [wave-dlam,wave+dlam]

    # Get nonlinear LDCs
    ldcs,mu,I0 = get_ldcs(min_w = wave_range[0], max_w = wave_range[1])
    u = ldcs[1:3] # Quadratic ldcs
    c = ldcs[6:10] # Nonlinear ldcs
    ldc_dict[ion] = {
        'wave_range':[],
        'ldc':{
            'quadratic':[],
            'nonlinear':[]
        }
    }
    ldc_dict[ion]['wave_range'] = wave_range
    ldc_dict[ion]['ldc']['quadratic'] = u
    ldc_dict[ion]['ldc']['nonlinear'] = c
    print(ion,wave_range,c)
    intensity = ldc_nonlinear(mu,c)
    ax.plot(mu,I0,label=ion,c='C%s'%(i+2),ls='--')
    #ax.plot(mu,intensity,c='C%s'%(i+2),ls='--')

ax.legend()

Fe II 1608 [1606.84155, 1610.05845] (np.float64(0.6251628826509708), np.float64(-2.149097236204477), np.float64(2.7756054800685486), np.float64(-0.22198859412568733))
Al II 1671 [1669.06926, 1672.41074] (np.float64(0.7206155788693114), np.float64(-2.2963978193024666), np.float64(2.735975115368417), np.float64(-0.11828954600978253))
Si II 1533 [1531.9165500000001, 1534.98345] (np.float64(0.22157257602393896), np.float64(0.0019306653786945402), np.float64(-1.5147571759481018), np.float64(2.3006965021670975))
Si II 1526 [1525.19328, 1528.24672] (np.float64(0.4026145598766748), np.float64(-0.7826022201528854), np.float64(-0.2932259636803548), np.float64(1.6994635373203792))
S I 1425 [1423.6049699999999, 1426.45503] (np.float64(-0.28146340001419656), np.float64(2.12965818421), np.float64(-5.227260361118754), np.float64(4.355839103250315))
S I 1433 [1431.84672, 1434.71328] (np.float64(-0.41481512604591325), np.float64(2.6576875151940085), np.float64(-6.090102191983093), np.float64(4.81505714

In [11]:
with open('ldcs.json','w') as file:
    json.dump(ldc_dict,file,indent=4)

In [12]:
# Fe II intensity fit
ldc_fe, mu_fe, I_fe = get_ldcs(min_w=1606.84155, max_w=1610.05845)
ldc_fe = ldc_fe[6:10]
intensity_fit = ldc_nonlinear(mu_fe,ldc_fe)

# Model LDCs from batman.ipynb
model = 4

match model:
    # BATMAN uniform
    case 1:
        ldcs = [0.47491752,-1.67688194,2.82185299,-0.09055813]
        label='BATMAN uniform'
    # BATMAN gaussian
    case 2:
        ldcs = [0.30023484,-1.74519917,2.95901681,0.02688968]
        label = 'BATMAN gaussian'
    # PyTransit uniform
    case 3:
        ldcs = [0.44333106,-2.11230645,2.56604699,0.07421745]
        label = 'PyTransit uniform'
    # PyTransit gaussian
    case 4:
        ldcs = [0.59993584,-2.17445874,2.75924478,-0.01660394]
        label = 'PyTransit gaussian'
        
mcmc_fit = ldc_nonlinear(mu_fe,ldcs)

fig,ax = plt.subplots(1,1,tight_layout=True)
pretty_axis(ax)
ax.plot(mu_fe,I_fe,lw=2,label='PHOENIX intensity')
ax.plot(mu_fe,intensity_fit,label='EJ15 fit')
ax.plot(mu_fe,mcmc_fit,label=label)
ax.legend(fontsize=14)
ax.set_xlabel(r'$\mu$',fontsize=18)
ax.set_ylabel(r'I($\mu$) / I(1)',fontsize=18)

Text(0, 0.5, 'I($\\mu$) / I(1)')

In [11]:
# MAKE INTENSITY PLOTS
plot_response = False
if plot_response:
    fig,ax = plt.subplots(1,1,tight_layout=True)
    pretty_axis(ax)
    ax.plot(S_wav,S_res)
    ax.set_xlabel("Wavelength [Å]",fontsize=16)
    ax.set_ylabel("Response",fontsize=16)
    ax.set_title("COS Segment A Response Function",fontsize=16)

plot_I0 = True
if plot_I0:
    fig,ax=plt.subplots(1,1,tight_layout=True)
    pretty_axis(ax)
    ax.plot(mu_a,I0_a,lw=3,label="Segment A")
    ax.plot(mu_b,I0_b,lw=3,label="Segment B")
    ax.plot(mu_a,intensity_nonlinear_a,ls='--',label="Seg A Non-linear")
    ax.plot(mu_b,intensity_nonlinear_b,ls='--',label="Seg B Non-linear")
    ax.set_xlabel(r"$\mu=\sqrt{1-r^2}$",fontsize=16)
    ax.set_ylabel("I / I(1)",fontsize=16)
    ax.set_title("PHOENIX Stellar Intensity Profile",fontsize=16)
    ax.legend(fontsize=13)

In [ ]:
np.save("COS_G160M_FLUXTAB_SENSITIVITY",arr=np.array([interp_wave,interp_sens]))
df = pd.DataFrame(data=np.array([interp_wave,interp_sens]).T)
df.to_csv("COS_G160M_FLUXTAB_SENSITIVITY.txt",header=False,index=False,sep=" ")

# Plot Al2 region

In [4]:
ldc_broad, mu_broad, I_broad = get_ldcs(1600,1760)
ldc_blue, mu_blue, I_blue = get_ldcs(1669,1670)
ldc_core, mu_core, I_core = get_ldcs(1670.48,1671)
ldc_red, mu_red, I_red = get_ldcs(1671,1672)

plt.figure()
for label,mu,I in zip(['broad','blue','core','red'],[mu_broad,mu_blue,mu_core,mu_red],[I_broad,I_blue,I_core,I_red]):
    plt.plot(mu,I,label=label)
plt.plot(mu_blue, I_blue/I_core, label=r'I$_{\mathrm{blue}}$ / I$_{\mathrm{core}}$')
plt.plot(mu_red, I_red/I_core, label=r'I$_{\mathrm{red}}$ / I$_{\mathrm{core}}$')
plt.xlabel(r'$\mu = \sqrt{1-r^2}$')
plt.ylabel(r'I($\mu$) / I(1)')
plt.legend()

# Make 2D intensity plot

In [16]:
from matplotlib.colors import LogNorm

response_file = "./COS_G160M_FLUXTAB_SENSITIVITY.txt"

infiles = glob("C:/Users/pabe9855/Desktop/Directories/KELT-20/PHOENIX/*ACES*SPECINT*.fits")
I_array = np.empty(shape=(len(infiles),25500,78))
i = 0
for file in infiles:
    with fits.open(file) as hdul:
        data = hdul[0].data
        mu = hdul[1].data
        CDELT1 = hdul[0].header['CDELT1']
        CRVAL1 = hdul[0].header['CRVAL1']
        
        wavelengths = np.arange(data.shape[1]) * CDELT1 + CRVAL1
        I = data.transpose()
        I_array[i] = I
        i += 1

I = np.mean(I_array,axis=0)
mask = (wavelengths > 1400) & (wavelengths < 1800)

wavelengths = wavelengths[mask]
I = I[mask]
I_norm = (I.T/I.T[-1]).T

# Plot PHOENIX intensity (not corrected for spherical approximation)
plt.figure()
im = plt.imshow(I,
               aspect='auto',
               origin='lower',
               norm=LogNorm(vmin=min(I.flatten()),vmax=max(I.flatten())),
               extent=(mu[0],mu[-1],wavelengths[0],wavelengths[-1]))
cb = plt.colorbar(im)
plt.axvline(x=0.08,c='k')

# Plot normalized PHOENIX intensity
plt.figure()
im = plt.imshow(I_norm,
           aspect='auto',
           origin='lower',
           norm=LogNorm(vmin=min(I_norm.flatten()),vmax=max(I_norm.flatten())),
           extent=(mu[0],mu[-1],wavelengths[0],wavelengths[-1]))
cb = plt.colorbar(im)

plt.axvline(x=0.08,c='k')

# Plot Al II directly from model (not including response curve)

In [14]:
line_center = 1393.75
al2_ind = np.argmin(np.abs(wavelengths-line_center))

plt.figure()
plt.plot(mu,I[al2_ind,:]/I[al2_ind,-1])

# average transits over Al II with different LDCs

In [5]:
import pytransit

In [48]:
# Transit parameters ( https://transit-timing.github.io/planets/KELT-20.html )
t_epoch = 2458312.58564 # Transit epoch in BJD
t_err = 0.00016 # Error on transit epoch
period = 3.47410024 # Orbital period in days
p_err = 0.0000061 # Error on period
a = 7.42 # semi-major axis [R_*]
e = 0 # Eccentricity 
inc = 1.503 # Radians
inc_deg = np.rad2deg(inc)

line_center = 1670.74
dlam = 300/3e5*line_center
wave = np.arange(line_center-dlam,line_center+dlam,step=0.1)

t_model = pytransit.RRModel('nonlinear')
x = np.linspace(-0.10,0.10,num=1000) # Phase array for pytransit model [days]
t_model.set_data(x)

transits = np.zeros(shape=(len(wave),len(x)))

for i in range(len(wave)-1):
    min_w = wave[i]
    max_w = wave[i+1]
    #print("Wave range: %0.2f - %0.2f"%(min_w,max_w))
    #print("Getting LDCs")
    ldc, mu, I = get_ldcs(min_w=min_w,max_w=max_w)
    ldc_nonlin = ldc[6:10]
    #print('Getting transit model')
    transits[i] = t_model.evaluate(k=0.13,t0=0.0,ldc=ldc_nonlin,a=a,e=e,i=inc,p=period)

transits = transits[0:-1]

ldc, mu, I = get_ldcs(min_w=min(wave),max_w=max(wave))
full_ldc = ldc[6:10]

transit_full = t_model.evaluate(k=0.13,t0=0.0,ldc=ldc_nonlin,a=a,e=e,i=inc,p=period)
transit_avg = np.average(transits,axis=0)

In [59]:
import astropy.units as u

min_ind = np.argmin(np.array([min(transit) for transit in transits]))
max_ind = np.argmax(np.array([max(transit) for transit in transits]))

d2hr = u.d.to(u.hr)

fig,ax = plt.subplots(2,1,sharex=True,figsize=(12,6),layout='constrained')
pretty_axis(ax)
ax[0].plot(x*d2hr,transit_full,lw=2,label='Constant LDC',c='k')
ax[0].plot(x*d2hr,transit_avg,lw=2,label='Avg LDC',c='r')

ax[0].fill_between(x=x*d2hr,y1=transits[min_ind],y2=transits[max_ind],color=('C0',0.3))
fig.supxlabel("Time from mid-transit [hr]",fontsize=18)
ax[0].set_ylabel(r'F$_{\rm{in}}$ / F$_{\rm{out}}$',fontsize=18)
ax[0].legend(fontsize=14)
ax[1].plot(x*d2hr,(transit_full-transit_avg)*100,lw=2)
ax[1].set_ylabel("Difference [%]",fontsize=18)
ax[0].set_xlim([-2,2])

(-2.0, 2.0)